# Notebook 2/

* SOFTWARE VERSION
* bedtools 2.26.0
* bismark 0.15.0
* blast 2.4.0+
* bowtie2 2.2.8
* bsmap 2.90 - needs python 2.x
* fastqc 0.11.5
* R 3.2.5
* RStudio
* samtools 0.1.19
* stacks 1.40
* trimmomatic 0.36
* multiqc 0.8
* md5sum 0.9.5

In [1]:
pwd

'/Users/hollie/Documents/Oly_Oyster_DNA_Methylation/Notebooks'

In [6]:
cd ..

/Users/hollie/Documents/Oly_Oyster_DNA_Methylation


In [10]:
!mkdir Data/MBD/BSmap_Output

mkdir: Data/MBD/BSmap_Output: File exists


In [11]:
%%bash
for file in Data/MBD/Clean_Joined_Data/zr1394_1_cleaned_all.fastq
do 
newname=${file##*/}
newname2=${newname/_cleaned_all.fastq/}
../BioInf_Programs/bsmap-2.74/bsmap -a Data/MBD/Clean_Joined_Data/$newname \
-d Data/Genome/Ostrea_lurida-Scaff-10k.fa \
-o Data/MBD/BSmap_Output/10K_bsmap_out_$newname2.sam \
-p 8;
done


BSMAP v2.74
Start at:  Thu Nov 17 10:47:08 2016

Input reference file: Data/Genome/Ostrea_lurida-Scaff-10k.fa 	(format: FASTA)
Load in 8733 db seqs, total size 131223850 bp. 2 secs passed
total_kmers: 43046721
Create seed table. 9 secs passed
max number of mismatches: read_length * 8% 	max gap size: 0
kmer cut-off ratio: 5e-07
max multi-hits: 100	max Ns: 5	seed size: 16	index interval: 4
quality cutoff: 0	base quality char: '!'
min fragment size:28	max fragemt size:500
start from read #1	end at read #4294967295
additional alignment: T in reads => C in reference
mapping strand: ++,-+
Single-end alignment(8 threads)
Input read file: Data/MBD/Clean_Joined_Data/zr1394_1_cleaned_all.fastq 	(format: FASTQ)
Output file: Data/MBD/BSmap_Output/10K_bsmap_out_zr1394_1.sam	 (format: SAM)
Thread #4: 	50000 reads finished. 17 secs passed
Thread #1: 	100000 reads finished. 17 secs passed
Thread #2: 	150000 reads finished. 18 secs passed
Thread #6: 	200000 reads finished. 18 secs passed
Thread #7: 	2

## Calculating Methyation Ratios

In [40]:
!/usr/bin/python ../BioInf_Programs/bsmap-2.74/methratio.py \
-d Data/Genome/Ostrea_lurida-Scaff-10k.fa \
-u -z -g \
-o Data/MBD/BSmap_Output/methratio_out_zr1394_1.txt \
-s ../BioInf_Programs/samtools-0.1.19/ \
Data/MBD/BSmap_Output/10K_bsmap_out_zr1394_1.sam

@ Thu Nov 17 13:54:38 2016: reading reference Data/Genome/Ostrea_lurida-Scaff-10k.fa ...
@ Thu Nov 17 13:54:44 2016: reading Data/MBD/BSmap_Output/10K_bsmap_out_zr1394_1.sam ...
[samopen] SAM header is present: 8733 sequences.
@ Thu Nov 17 13:56:25 2016: combining CpG methylation from both strands ...
@ Thu Nov 17 13:56:37 2016: writing Data/MBD/BSmap_Output/methratio_out_zr1394_1.txt ...
@ Thu Nov 17 13:57:38 2016: done.
total 2099435 valid mappings, 1868410 covered cytosines, average coverage: 4.16 fold.


# Converting methratio files (% methylation) into format for methylkit

* Add scripts for format conversion

In [47]:
!mkdir Scripts

mkdir: Scripts: File exists


In [48]:
%%bash
echo \
'#!/usr/bin/awk -f

!awk {if ($8 >= 3) print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12}' > Scripts/mr3x.awk

In [49]:
!head Scripts/mr3x.awk

#!/usr/bin/awk -f

!awk {if ($8 >= 3) print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12}


In [50]:
%%bash
echo \
'#!/usr/bin/awk -f

BEGIN{ print "chr.Base\tchr\tbase\tstrand\tcoverage\tfreqC\tfreqT" }
{
	if ($3 == "+") {
		strand="F"
 	} else {
		strand="R"
	}

	FC=($7/$8)*100
	FT=(1-($7/$8))*100
	chrbase=$1"."$2
	printf "%s\t%s\t%s\t%s\t%d\t%.2f\t%.2f\n",
		chrbase, $1, $2, strand, $8, FC, FT
}' > Scripts/mr_gg.awk.sh

In [51]:
!cat Scripts/mr_gg.awk.sh

#!/usr/bin/awk -f

BEGIN{ print "chr.Base\tchr\tbase\tstrand\tcoverage\tfreqC\tfreqT" }
{
	if ($3 == "+") {
		strand="F"
 	} else {
		strand="R"
	}

	FC=($7/$8)*100
	FT=(1-($7/$8))*100
	chrbase=$1"."$2
	printf "%s\t%s\t%s\t%s\t%d\t%.2f\t%.2f\n",
		chrbase, $1, $2, strand, $8, FC, FT
}


In [53]:
%%bash
for file in Data/MBD/BSmap_Output/methratio_out_*.txt
do
newname=${file##*/}
newname2=${newname/methratio_out_/}
newname3=${newname2/.txt/}     
echo $newname3
grep "CG" <methratio_out_$newname3.txt> methratio_out_$newname3.CG.txt
awk -f Scripts/mr3x.awk methratio_out_$newname3.CG.txt > Data/MBD/BSmap_Output/mr3x.$newname3.txt
awk -f Scripts/mr_gg.awk.sh mr3x.$newname3.txt > Data/MBD/BSmap_Output/mkfmt_$newname3.txt;
done

zr1394_1
